<p align="center">
<img src="../assets/images/dtlogo.png" alt="Duckietown Logo" width="50%">
</p>

# Duckiebot Components - The LEDs

Duckiebots have four LEDs, positioned similarly to the head and tail lights on a car.

<p align="center">
<img src="../assets/images/leds_layout.png" alt="Duckiebot LED Layout" width="66%">
</p>

You have probably noticed the LEDs changing color to communicate the state of your Duckiebot while booting or running demos. LEDs as actuators on a Duckiebot can be used for many purposes, including

* **Lighting** the driving environment
* **Indicating** what mode or mission the Duckiebot is running
* **Communicating** state changes in the controller
* **Signaling** upcoming turns or other navigation plans
* **Expressing** character and personality

In this Duckietown Learning Experience, we will dive into what's going on behind the scenes of the LED controller and explore ways you can update the LEDs with your own custom messages.

Let's practice instructing the LEDs on your Duckiebot - changing their color, blinking them at different frequencies, and displaying your own patterns.

---

## Step 1: Using the LED widget

While setting up your Duckiebot, you were able to control the LEDs using the `led_control` command. Try this again by running

    dts duckiebot led_control <your_duckiebot>

This will bring up the LED control widget, which you can use to send commands to the LEDs.

Try updating the LED color and intensity to see how your Duckiebot responds.

<p align="center">
<img src="../assets/images/led_widget.png" alt="🔴 TODO: Widget image" width="33%">
</p>

## Step 2: Understanding the LED message

The LED tool you just used sets the state of the LEDs on your Duckiebot using a custom Duckietown ROS message: `LEDPattern`. The pattern includes a list of led values that update the color, intensity, and frequency of each individual LED.

By sending this message to the *led_emitter* node running on the Duckiebot, we can instruct this node to update the LEDs to the states in the message and display our pattern.

You can see the full set of Duckietown ROS messages in the [`msg` directory](https://github.com/duckietown/dt-ros-commons/tree/daffy/packages/duckietown_msgs/msg) of the `dt-ros-commons` respository, but for now we'll just look at the `LEDPattern` message.

**LED Pattern**

``` bash
# LED Pattern Message
# This defines a pattern for all 4 LEDs

Header header
string[]  color_list
int8[]    color_mask
float32   frequency
int8[]    frequency_mask
std_msgs/ColorRGBA[]  rgb_vals
```

### What are these messages communicating?

This message is used to communicate with the *led_emitter* and *led_driver* nodes.  The *led_emitter* takes an `LEDPattern` and constructs a set of *RGBA* values for the LEDs to match the pattern.  It then sends those values to the *led_driver* to update the state of each individual LED.  We will cover both of these nodes in more depth in the next notebook.

First, let's break down the important pieces of the `duckietown_msgs/LEDPattern` message and answer some key questions ...

**What is the Header?**

The ROS `header` passes along some standard metadata with the message, including the timestamp at which it was sent and a unique message ID number that increases consecutively with every message published.  You can find more details on the ROS documentation for the [std_msgs/Header message](http://docs.ros.org/en/melodic/api/std_msgs/html/msg/Header.html).

**How are we setting the color?**

The color of each individual LED is defined using the `R, G, B` components of the `std_msgs/ColorRGBA[] rgb_vals` in the `LEDPattern` message.  Each color component is a `float32` value from 0.0-1.0 that is mapped to the 0-255 color range.  For example, a red LED would be represented as `{r: 1.0, g: 0.0, b: 0.0, a: 1.0}`  The *led_driver* node is given only this portion of the `LEDPattern` message.

When defining an LED pattern rather than commanding individual LEDs, we can use `color_list` to define the LED colors as a list of strings such as `['green','yellow','pink','orange','blue']`.  When given a full pattern, the *led_emitter* node translates this to the required `rgb_vals` for the *led_driver*.

**How are we setting the frequency?**

Each LED blinks at a frequency defined by the `float32 frequency` in the `LEDPattern` message.  The frequency can be applied to individual LEDs using the frequency mask, which updates only the LED indices set to one. For example, the mask `[1, 0, 0, 0, 1]` would cause only the front left and front right LEDs to blink at the given frequency.

**Which LED is which?**

The lists in the `LEDPattern` message map to the following LED order

| Index | Position |
| ----- | -------- |
| 0 | Front left |
| 1 | Rear left |
| 2 | Top |
| 3 | Rear right |
| 4 | Front right |

As you can see, we want to update the LEDs at indices `[0, 1, 3, 4]` to control the head and tail lights.

## Next Up: Sending Your Own LED Messages

Now that you understand how we communicate new LED states to the Duckiebot's LEDs, continue on to the [next notebook](./02_Sending-LED-Messages.ipynb) to dive into the architecture of the LED nodes and send your own `LEDPattern` messages.